# Assignment 2: Analyzing large datasets with Spark.


For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v $(pwd):/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [218]:
### Write your code here

import re
from pyspark import SparkContext

In [219]:
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_33/3509690429.py:5 

### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [ ]:
### Write your code here

book_43 = sc.textFile('data/43.txt')

### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [ ]:
### Write your code here

lines_43 = book_43.count()
lines_43

### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [ ]:
### Write your code here

test_line = "This is an example of that contains 234 and a dash-containing number"

def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()

clean_split_line(test_line)

### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [ ]:
### Write your code here
book_43_words = book_43.flatMap(clean_split_line)
book_43_words.count()

### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [ ]:
### Write your code here

book_43_words = book_43_words.map(lambda x: x.lower())

Q6 = book_43_words.map(lambda x: (x,1))
Q6 = Q6.reduceByKey(lambda x,y: x+y)
uniqueWords = Q6.filter(lambda x: x[1] == 1)
uniqueWords.count()

### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [ ]:
### Write your code here

book_43_counts = Q6
book_43_counts.collect()

### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [ ]:
### Write your code here
Q8 = Q6.map(lambda x: tuple(reversed(x)))
book_43_counts = Q8.sortByKey(ascending = 0)
book_43_counts.take(20)


### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [ ]:
### Write your code here
import sklearn.feature_extraction
from sklearn.feature_extraction import _stop_words
stop_words = sc.parallelize(list(_stop_words.ENGLISH_STOP_WORDS))
stop_words.collect()

### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [ ]:
### Write your code here
# https://stackoverflow.com/questions/26214112/filter-based-on-another-rdd-in-spark
book_43_counts_filtered = book_43_counts.map(lambda x: tuple(reversed(x))).filter(lambda x: x[0] not in _stop_words.ENGLISH_STOP_WORDS)
book_43_counts_filtered.collect()

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [ ]:
### Write your code here

book_43_counts_filtered.map(lambda x: x[1]).sum()

### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [220]:
### Write your code here
def process_RDD(file):
    # Read textfile as RDD
    t = sc.textFile(file)
    
    # Clean and split line
    t = t.map(lambda x: re.sub('\d+', '', x))
    t = t.map(lambda x: re.sub('[\W]+', ' ', x))
    t = t.flatMap(lambda x: x.split())
    
    # Convert to lowercase and count words
    t = t.map(lambda x: x.lower())
    t = t.map(lambda x: (x,1))
    t.collect()
    t = t.reduceByKey(lambda x,y: x+y)
    
    # Filter stop words
    t = t.filter(lambda x: x[0] not in _stop_words.ENGLISH_STOP_WORDS)
    
    return t

### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [243]:
### Write your code here

book_84_counts_filtered = process_RDD('data/84.txt')
book_398_counts_filtered = process_RDD('data/398.txt')
book_3296_counts_filtered = process_RDD('data/3296.txt')

print("Book_84:", book_84_counts_filtered.count(),"| Book_398:",book_398_counts_filtered.count(),"| Book_3296:", book_3296_counts_filtered.count())

Book_84: 7016 | Book_398: 2421 | Book_3296: 7293


### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [222]:
### Write your code here

compare_84_3296 = book_84_counts_filtered.join(book_3296_counts_filtered)
compare_84_398 = book_84_counts_filtered.join(book_398_counts_filtered)
print("Books 84 to 3296:", compare_84_3296.count())
print("Books 84 to 398:", compare_84_398.count())

Books 84 to 3296: 3608
Books 84 to 398: 1691


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

#### Write your answer here

Although counting the # of shared words can be a useful metric, in many cases, like Q14, it is proven to be ineffective at determining topic similarity, since 398 and 3296 are both religious texts,

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


In [223]:
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]

from scipy.spatial.distance import euclidean 

print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")

The Euclidean distance between A and B is: 3.1622776601683795
The Euclidean distance between A and C is: 14.0
The Euclidean distance between B and C is: 16.431676725154983


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [224]:
book_43_counts_filtered = process_RDD('data/43.txt')

In [225]:
### Write your code here

common_words = book_84_counts_filtered.join(book_3296_counts_filtered).join(book_398_counts_filtered).join(book_43_counts_filtered)
common_words = common_words.keys().collect()

# Create matrix

matrix = np.zeros([4, len(common_words)])

# Filter out common words for each column
book_43_row = book_43_counts_filtered.filter(lambda x: x[0] in common_words)
book_84_row = book_84_counts_filtered.filter(lambda x: x[0] in common_words)
book_398_row = book_398_counts_filtered.filter(lambda x: x[0] in common_words)
book_3296_row = book_3296_counts_filtered.filter(lambda x: x[0] in common_words)

# Sort rows before adding to matrix

book_43_row = book_43_row.sortByKey()
book_84_row = book_84_row.sortByKey()
book_398_row = book_398_row.sortByKey()
book_3296_row = book_3296_row.sortByKey()

# Add rows to matrix

matrix[0,:] = book_43_row.flatMap(lambda x: [x[1]]).collect()
matrix[1,:] = book_84_row.flatMap(lambda x: [x[1]]).collect()
matrix[2,:] = book_398_row.flatMap(lambda x: [x[1]]).collect()
matrix[3,:] = book_3296_row.flatMap(lambda x: [x[1]]).collect()

matrix


array([[ 1.,  3.,  2., ...,  5.,  3.,  1.],
       [ 1.,  9.,  4., ..., 30.,  9.,  1.],
       [ 1.,  5., 16., ...,  1.,  2.,  1.],
       [16., 36.,  3., ..., 15.,  4.,  1.]])

### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?


In [226]:
### Write your code here

print("Euclidean distance between `book_398` and `book_3296`:", euclidean(matrix[2,:], matrix[3,:]))
print("Euclidean distance between `book_84` and `book_398`:", euclidean(matrix[1,:], matrix[2,:]))

Euclidean distance between `book_398` and `book_3296`: 1156.6628722320086
Euclidean distance between `book_84` and `book_398`: 751.6688100486809


#### Write your justification here

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

#### Write your answer here

If we were to show similarity using a common word matrix, then it makes sense to limit the number of books used in the matrix. As the amount of books increases, the amount of common words decreases, and similarities between any two given books would be harder to find. An alternative is to include, rather than exclude words that are common among pairs of books.

## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [227]:
### Write your code here

from pyspark.sql import SparkSession

spark = SparkSession(sc)
flight_info = spark.read.csv('data/flight_info.csv', header=True)
flight_info.count()

450017

### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [228]:
### Write your code here

# Pandas
flight_info.select("UniqueCarrier").distinct().show()

+-------------+
|UniqueCarrier|
+-------------+
|           UA|
|           NK|
|           AA|
|           EV|
|           B6|
|           DL|
|           OO|
|           F9|
|           HA|
|           WN|
|           AS|
|           VX|
+-------------+



### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [229]:
### Write your code here

flight_delays = flight_info.filter(flight_info["DepTime"] > flight_info["CRSDepTime"])
flight_delays = flight_delays.groupby("UniqueCarrier").count()
flight_delays = flight_delays.sort("count", ascending=0)
flight_delays.show()

+-------------+-----+
|UniqueCarrier|count|
+-------------+-----+
|           WN|54557|
|           DL|28962|
|           AA|26291|
|           UA|19594|
|           OO|17924|
|           EV|12340|
|           B6|10406|
|           AS| 4966|
|           NK| 4435|
|           F9| 3181|
|           VX| 2871|
|           HA| 2265|
+-------------+-----+



### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [230]:
### Write your code here

airlines_info = spark.read.csv("data/airlines.csv")

airlines = airlines_info.join(flight_info,  flight_info["UniqueCarrier"] == airlines_info["_c3"])
airlines = airlines.filter("DepTime > CRSDepTime")

airlines = airlines.groupby("UniqueCarrier","_c1").count()
airlines = airlines.sort("count", ascending=False)


airlines.show()

+-------------+--------------------+-----+
|UniqueCarrier|                 _c1|count|
+-------------+--------------------+-----+
|           WN|  Southwest Airlines|54557|
|           DL|     Delta Air Lines|28962|
|           AA|   American Airlines|26291|
|           UA|     United Airlines|19594|
|           OO|             SkyWest|17924|
|           EV|Atlantic Southeas...|12340|
|           B6|     JetBlue Airways|10406|
|           AS|     Alaska Airlines| 4966|
|           NK|     Spirit Airlines| 4435|
|           F9|   Frontier Airlines| 3181|
|           VX|      Virgin America| 2871|
|           HA|   Hawaiian Airlines| 2265|
+-------------+--------------------+-----+



### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [231]:
### Write your code here
delays_per_company = flight_info.filter("DepTime > CRSDepTime").groupby("UniqueCarrier", "DayOfWeek").count().sort("UniqueCarrier", "DayOfWeek")
delays_per_company.show()

+-------------+---------+-----+
|UniqueCarrier|DayOfWeek|count|
+-------------+---------+-----+
|           AA|        1| 5117|
|           AA|        2| 3688|
|           AA|        3| 2941|
|           AA|        4| 3575|
|           AA|        5| 3525|
|           AA|        6| 2800|
|           AA|        7| 4645|
|           AS|        1|  872|
|           AS|        2|  669|
|           AS|        3|  618|
|           AS|        4|  728|
|           AS|        5|  630|
|           AS|        6|  517|
|           AS|        7|  932|
|           B6|        1| 1892|
|           B6|        2| 1787|
|           B6|        3| 1253|
|           B6|        4| 1254|
|           B6|        5| 1309|
|           B6|        6| 1027|
+-------------+---------+-----+
only showing top 20 rows



### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [232]:
### Write your code here

delays_per_company_total = flight_info.groupby("UniqueCarrier", "DayOfWeek").count().sort("UniqueCarrier", "DayOfWeek")

normalized = delays_per_company.join(delays_per_company_total, ["UniqueCarrier", "DayOfWeek"]).withColumn("NormalizedDelays", delays_per_company["count"] / delays_per_company_total["count"])
normalized = normalized.select("UniqueCarrier", "DayOfWeek", "NormalizedDelays")
normalized.show()


+-------------+---------+-------------------+
|UniqueCarrier|DayOfWeek|   NormalizedDelays|
+-------------+---------+-------------------+
|           F9|        6| 0.3603238866396761|
|           OO|        3| 0.3355342136854742|
|           EV|        6| 0.2974967907573813|
|           F9|        1| 0.5031847133757962|
|           WN|        4| 0.5567082085915199|
|           AA|        1| 0.4206329634196465|
|           UA|        7| 0.4904064058014806|
|           OO|        4|0.38484179543782193|
|           NK|        1| 0.4169124877089479|
|           B6|        6| 0.3583391486392184|
|           EV|        1| 0.4186756669881067|
|           F9|        2| 0.4022801302931596|
|           AA|        3|0.30285243538255585|
|           UA|        3|0.45242933678182157|
|           EV|        7| 0.3808161708619375|
|           B6|        7|0.48085758039816234|
|           UA|        6| 0.3950864878415643|
|           B6|        3|0.39601769911504425|
|           B6|        4|0.3827838

### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [233]:
### Write your code here



delays_per_company_total = flight_info.groupby("UniqueCarrier", "DayOfWeek").count().sort("UniqueCarrier", "DayOfWeek")

normalized = delays_per_company.join(delays_per_company_total, ["UniqueCarrier", "DayOfWeek"]).withColumn("NormalizedDelays", delays_per_company["count"] / delays_per_company_total["count"])
normalized = normalized.select("UniqueCarrier", "DayOfWeek", "NormalizedDelays")
normalized.show()

%time

+-------------+---------+-------------------+
|UniqueCarrier|DayOfWeek|   NormalizedDelays|
+-------------+---------+-------------------+
|           F9|        6| 0.3603238866396761|
|           OO|        3| 0.3355342136854742|
|           EV|        6| 0.2974967907573813|
|           F9|        1| 0.5031847133757962|
|           WN|        4| 0.5567082085915199|
|           AA|        1| 0.4206329634196465|
|           UA|        7| 0.4904064058014806|
|           OO|        4|0.38484179543782193|
|           NK|        1| 0.4169124877089479|
|           B6|        6| 0.3583391486392184|
|           EV|        1| 0.4186756669881067|
|           F9|        2| 0.4022801302931596|
|           AA|        3|0.30285243538255585|
|           UA|        3|0.45242933678182157|
|           EV|        7| 0.3808161708619375|
|           B6|        7|0.48085758039816234|
|           UA|        6| 0.3950864878415643|
|           B6|        3|0.39601769911504425|
|           B6|        4|0.3827838

In [234]:
# Hand recorded times, in microseconds
# I'm not sure if I'm doing this wrong, it seems too fast

recorded_times = [4.53, 4.29, 4.77, 4.53, 4.53, 5.01, 4.29, 4.77, 5.01]
avg = sum(recorded_times)/len(recorded_times)
print("Average time:",avg,"microseconds")

Average time: 4.636666666666667 microseconds


### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [235]:
### Write your code here

### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [237]:
### Write your code here

import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import metrics, datasets, model_selection
from sklearn.linear_model import LinearRegression

# Generate Departure/Arrival Delay Columns

flight_delay_data = flight_info.select("DepTime", "CRSDepTime", "ArrTime", "CRSArrTime")
flight_delay_data = flight_delay_data.where("DepTime > CRSDepTime")
flight_delay_data = flight_delay_data.withColumn("Departure Delay", flight_info.DepTime - flight_info.CRSDepTime)
flight_delay_data = flight_delay_data.withColumn("Arrival Delay", flight_info.ArrTime - flight_info.CRSArrTime)
flight_delay_data = flight_delay_data.select("Departure Delay", "Arrival Delay")

# Build ML model, using Pandas DataFrame & Sklearn, not sure how to use Spark yet

model = LinearRegression()
df = flight_delay_data.toPandas()
df = df.dropna() # Remove rows w/ null values

# Fit model, first split original dataset into train/test datasets, then split the test dataset into valid/test datasets
# 80% train, 10% valid, 10% test

X = df["Departure Delay"].to_frame()
y = df["Arrival Delay"].to_frame()

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

model.fit(X_train, y_train)
val_predictions = model.predict(X_test)

print('Model MAE Test Score:', metrics.mean_absolute_error(y_test, val_predictions))

Model MAE Test Score: 133.5962877343281
